In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS1T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment3_WS1_has_duration.csv")

In [3]:
df3_WS1T.head(n=1)


,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1161800|550469|842233,Submit,Submit,18/11/2019 22:13:27,GUEST,Student,9,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,AF,AF,NaN,NaN,N,6 days 14 hours 58 minutes 45 seconds,572325.0


In [4]:
numOfWorkflowProcesses = df3_WS1T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))


In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS1T[df3_WS1T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS1T[df3_WS1T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))


In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS1T[df3_WS1T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Send for Assessment']
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS1T[df3_WS1T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [13]:
s3_t1=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s3_t1.head(n=10)



,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,36,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Submit}
1,2,31,10 days 15 hours 23 minutes 16 seconds,4.0 days 22.0 hours 39.0 minutes 2.0 seconds,37.1%,"{Send for Assessment, Put Triage On-Hold, AR -..."
2,3,15,17 days 20 hours 35 minutes 6 seconds,11.0 days 10.0 hours 36.0 minutes 33.0 seconds,24.44%,"{Send for Assessment, AR - Created - previous ..."
3,4,75,14 days 14 hours 41 minutes 33 seconds,10.0 days 17.0 hours 32.0 minutes 0.0 seconds,23.0%,"{Send for Assessment, AR - Created - previous ..."
4,5,86,11 days 21 hours 17 minutes 30 seconds,8.0 days 7.0 hours 7.0 minutes 46.0 seconds,17.44%,"{AR - Satisfied - Other, AR - Created - furthe..."
5,6,116,14 days 16 hours 57 minutes 39 seconds,8.0 days 4.0 hours 4.0 minutes 16.5 seconds,15.37%,"{AR - Reviewed - further information, AR - Sat..."
6,7,113,17 days 17 hours 16 minutes 18 seconds,12.0 days 17.0 hours 15.0 minutes 1.0 seconds,13.27%,"{Assessment Decision - No Decision, AR - Creat..."
7,8,91,15 days 0 hours 32 minutes 30 seconds,11.0 days 6.0 hours 32.0 minutes 18.0 seconds,11.54%,"{OC - Reviewed - Other, AR - Satisfied - Other..."
8,9,49,25 days 18 hours 49 minutes 17 seconds,16.0 days 20.0 hours 18.0 minutes 7.0 seconds,10.43%,"{OC - Reviewed - Other, AR - Satisfied - Other..."
9,10,58,21 days 1 hours 31 minutes 16 seconds,16.0 days 1.0 hours 31.0 minutes 25.0 seconds,8.79%,"{OC - Reviewed - Other, AR - Requested - sylla..."


In [14]:
s3_t1.to_csv('segment3_WS1_table.csv', index=False)
